# 2. Open API 활용

### 공공데이터 포털사이트 사용하기
1. 공공데이터 포털사이트 회원가입/로그인 
2. 데이터 검색
3. API 사용 요청, 키 발급
4. API 매뉴얼 확인
5. API 연동 개발 및 테스트

In [1]:
import requests
import xmltodict


### 보건복지부 - 코로나 19 감염현황조회서비스(XML)

In [2]:
serviceKey = '7PkCtZVnKSDI%2BMc4iNiOu%2B0RTpLnRtHbmR52PF9yt8wVKlJOEJ4Yomdfw%2BsvoYRcow3dMaIK%2B0HWigjaKh8ZoQ%3D%3D'

url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey={}&pageNo=1&numOfRows=10&startCreateDt=20210601&endCreateDt=20210816'.format(serviceKey)
url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=7PkCtZVnKSDI%2BMc4iNiOu%2B0RTpLnRtHbmR52PF9yt8wVKlJOEJ4Yomdfw%2BsvoYRcow3dMaIK%2B0HWigjaKh8ZoQ%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20210601&endCreateDt=20210816'

In [3]:
res=requests.get(url)
res

<Response [200]>

In [4]:
res.text[:100]

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode>'

In [5]:
temp=xmltodict.parse(res.text)
type(temp)

collections.OrderedDict

In [6]:
import json
result = json.loads(json.dumps(temp)) # dumps : ordereddict -> string // loads string -> dict
#result

In [7]:
import pandas as pd
# 숙제
# STATE_DT(기준일)
# DECIDE_CNT(확진자수)
# CLEAR_CNT(격리해제수)
# DEATH_CNT(사망자수)
# CARE_CNT(치료자수)
state = list()
decide = list()
clear = list()
death = list()
care = list()
for i in result['response']['body']['items']['item'] :
    state.append(i['stateDt'])
    decide.append(i['decideCnt'])
    clear.append(i['clearCnt'])
    death.append(i['deathCnt'])
    care.append(i['careCnt'])

In [8]:
df_data = {
    '기준일': state,
    '확진자수' : decide,
    '격리해제수' : clear,
    '사망자수' : death,
    '치료자수' : care
}
df = pd.DataFrame(df_data)

df

,기준일,확진자수,격리해제수,사망자수,치료자수
0,20210816,225474,196198,2167,27109
1,20210815,223919,195103,2156,26660
2,20210814,222103,193778,2148,26177
3,20210813,220175,192248,2144,25783
4,20210812,218186,190536,2138,25512
...,...,...,...,...,...
72,20210605,143593,133766,1971,7856
73,20210604,142849,133287,1969,7593
74,20210603,142154,132697,1968,7489
75,20210602,141473,132067,1965,7441


### 한국 관광 공사 - 관광정보서비스


In [9]:
url = 'http://api.visitkorea.or.kr/openapi/service/rest/KorService/areaCode?ServiceKey={}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=TestApp&_type=json'.format(serviceKey)
url

'http://api.visitkorea.or.kr/openapi/service/rest/KorService/areaCode?ServiceKey=7PkCtZVnKSDI%2BMc4iNiOu%2B0RTpLnRtHbmR52PF9yt8wVKlJOEJ4Yomdfw%2BsvoYRcow3dMaIK%2B0HWigjaKh8ZoQ%3D%3D&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=TestApp&_type=json'

In [10]:
res = requests.get(url)
res

<Response [200]>

In [11]:
res.text

'{"response":{"header":{"resultCode":"0000","resultMsg":"OK"},"body":{"items":{"item":[{"code":1,"name":"서울","rnum":1},{"code":2,"name":"인천","rnum":2},{"code":3,"name":"대전","rnum":3},{"code":4,"name":"대구","rnum":4},{"code":5,"name":"광주","rnum":5},{"code":6,"name":"부산","rnum":6},{"code":7,"name":"울산","rnum":7},{"code":8,"name":"세종특별자치시","rnum":8},{"code":31,"name":"경기도","rnum":9},{"code":32,"name":"강원도","rnum":10}]},"numOfRows":10,"pageNo":1,"totalCount":17}}}'

In [12]:
data = res.json()
data

{'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'},
  'body': {'items': {'item': [{'code': 1, 'name': '서울', 'rnum': 1},
     {'code': 2, 'name': '인천', 'rnum': 2},
     {'code': 3, 'name': '대전', 'rnum': 3},
     {'code': 4, 'name': '대구', 'rnum': 4},
     {'code': 5, 'name': '광주', 'rnum': 5},
     {'code': 6, 'name': '부산', 'rnum': 6},
     {'code': 7, 'name': '울산', 'rnum': 7},
     {'code': 8, 'name': '세종특별자치시', 'rnum': 8},
     {'code': 31, 'name': '경기도', 'rnum': 9},
     {'code': 32, 'name': '강원도', 'rnum': 10}]},
   'numOfRows': 10,
   'pageNo': 1,
   'totalCount': 17}}}

In [13]:
numOfRows = data['response']['body']['numOfRows']
totalCount = data['response']['body']['totalCount']
numOfRows, totalCount

(10, 17)

In [14]:
queryCount = totalCount // numOfRows

if totalCount % numOfRows > 0:
    queryCount += 1

queryCount

2

In [15]:
code = []
name = []
rnum = []

for i in range(queryCount):
    pageNo = i + 1
    url = 'http://api.visitkorea.or.kr/openapi/service/rest/KorService/areaCode?ServiceKey={}&numOfRows=10&pageNo={}&MobileOS=ETC&MobileApp=TestApp&_type=json'.format(serviceKey, pageNo)
    requests.get(url)
    data = res.json()
    
    items = data['response']['body']['items']['item']
    
    for item in items:
        code.append(item['code'])
        name.append(item['name'])
        rnum.append(item['rnum'])
        

name

['서울',
 '인천',
 '대전',
 '대구',
 '광주',
 '부산',
 '울산',
 '세종특별자치시',
 '경기도',
 '강원도',
 '서울',
 '인천',
 '대전',
 '대구',
 '광주',
 '부산',
 '울산',
 '세종특별자치시',
 '경기도',
 '강원도']

In [16]:
df_data = {
    'code' = code,
    'name' = name,
    'rnum' = rnum
}

df = pd.DataFrame(data= df_data)
df

SyntaxError: invalid syntax (<ipython-input-16-8e41369123d8>, line 2)